In [1]:
# current future warning with pandas
import warnings
warnings.simplefilter(action ='ignore', category = FutureWarning)

import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module 
import scipy.stats as sts
import matplotlib.pyplot as plt  
import backtesting as bt
import talib
import csv

In [2]:
# SE30 Sweden 30 Index
SE30_index = pd.read_csv("./Indexies/SE30_5M_HKT.csv")
SE30_index.index = pd.to_datetime(SE30_index['Date'])
SE30_index = SE30_index.drop(columns=['Date'])

# US2000 US smallcap 2000 Index
US2000_index = pd.read_csv("./Indexies/US2000_5M_HKT.csv")
US2000_index.index = pd.to_datetime(US2000_index['Date'])
US2000_index = US2000_index.drop(columns=['Date'])

joined_index = pd.concat([SE30_index, US2000_index], axis=1, join='inner')
joined_index.dropna(inplace=True)
joined_index.columns = ['Open1', 'High1', 'Low1', 'Close1', 'Volume1', 'Open2', 'High2', 'Low2', 'Close2', 'Volume2']
SE30_index = joined_index[['Open1', 'High1', 'Low1', 'Close1', 'Volume1']]
SE30_index.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
US2000_index = joined_index[['Open2', 'High2', 'Low2', 'Close2', 'Volume2']]
US2000_index.columns = ['Open', 'High', 'Low', 'Close', 'Volume']


In [3]:
join_index = pd.concat([SE30_index['Close'], US2000_index['Close']], axis="columns")
join_index.dropna(inplace=True)
join_index.columns = ['SE30', 'US2000']
print(join_index)
SE = join_index['SE30']*20
US2000 = join_index['US2000']*21
ratio = US2000 / SE
print(ratio)

                        SE30   US2000
Date                                 
2020-04-20 15:00:00  1544.63  1217.84
2020-04-20 15:05:00  1541.87  1218.29
2020-04-20 15:10:00  1537.75  1216.99
2020-04-20 15:15:00  1531.57  1215.19
2020-04-20 15:20:00  1533.35  1216.39
...                      ...      ...
2022-06-29 20:40:00  1903.71  1736.33
2022-06-29 20:45:00  1904.31  1735.68
2022-06-29 20:50:00  1904.30  1736.33
2022-06-29 20:55:00  1904.50  1735.43
2022-06-29 21:00:00  1904.37  1737.13

[55016 rows x 2 columns]
Date
2020-04-20 15:00:00    0.827857
2020-04-20 15:05:00    0.829645
2020-04-20 15:10:00    0.830980
2020-04-20 15:15:00    0.833099
2020-04-20 15:20:00    0.832954
                         ...   
2022-06-29 20:40:00    0.957681
2022-06-29 20:45:00    0.957021
2022-06-29 20:50:00    0.957384
2022-06-29 20:55:00    0.956787
2022-06-29 21:00:00    0.957790
Length: 55016, dtype: float64


In [4]:
SE30_index ['Ratio'] = ratio
US2000_index ['Ratio'] = ratio

/var/folders/qt/h3qgx4hn4j9cdtgy975g7g2r0000gn/T/ipykernel_4128/2444217905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SE30_index ['Ratio'] = ratio
/var/folders/qt/h3qgx4hn4j9cdtgy975g7g2r0000gn/T/ipykernel_4128/2444217905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US2000_index ['Ratio'] = ratio


In [5]:
SE30_index

,Open,High,Low,Close,Volume,Ratio
Date,,,,,,
2020-04-20 15:00:00,1551.22,1551.22,1542.25,1544.63,258,0.827857
2020-04-20 15:05:00,1544.75,1547.25,1540.12,1541.87,238,0.829645
2020-04-20 15:10:00,1541.75,1542.37,1537.00,1537.75,172,0.830980
2020-04-20 15:15:00,1537.87,1537.87,1531.32,1531.57,171,0.833099
2020-04-20 15:20:00,1531.44,1534.11,1529.79,1533.35,191,0.832954
...,...,...,...,...,...,...
2022-06-29 20:40:00,1901.98,1903.96,1901.36,1903.71,129,0.957681
2022-06-29 20:45:00,1903.46,1905.43,1902.71,1904.31,137,0.957021
2022-06-29 20:50:00,1904.31,1905.44,1903.93,1904.30,94,0.957384


In [6]:
ratio_copy = pd.concat([ratio, ratio, ratio, ratio, ratio], axis="columns")

# for backtesting.py requirements
ratio_copy.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
pd.DataFrame(ratio_copy)


,Open,High,Low,Close,Volume
Date,,,,,
2020-04-20 15:00:00,0.827857,0.827857,0.827857,0.827857,0.827857
2020-04-20 15:05:00,0.829645,0.829645,0.829645,0.829645,0.829645
2020-04-20 15:10:00,0.830980,0.830980,0.830980,0.830980,0.830980
2020-04-20 15:15:00,0.833099,0.833099,0.833099,0.833099,0.833099
2020-04-20 15:20:00,0.832954,0.832954,0.832954,0.832954,0.832954
...,...,...,...,...,...
2022-06-29 20:40:00,0.957681,0.957681,0.957681,0.957681,0.957681
2022-06-29 20:45:00,0.957021,0.957021,0.957021,0.957021,0.957021
2022-06-29 20:50:00,0.957384,0.957384,0.957384,0.957384,0.957384


# Adding BBands Indicator

In [7]:
ratio_copy = pd.DataFrame()
#print(SE30_index['Ratio'])
temp = pd.DataFrame()
temp['Close'] = SE30_index['Ratio'].dropna()
#print(temp.head())
ratio_copy['Lowerband'], ratio_copy['Middleband'], ratio_copy['Upperband'] = talib.BBANDS(temp['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
print(ratio_copy)
print(ratio_copy['Lowerband'].isna().sum())

                     Lowerband  Middleband  Upperband
Date                                                 
2020-04-20 15:00:00        NaN         NaN        NaN
2020-04-20 15:05:00        NaN         NaN        NaN
2020-04-20 15:10:00        NaN         NaN        NaN
2020-04-20 15:15:00        NaN         NaN        NaN
2020-04-20 15:20:00        NaN         NaN        NaN
...                        ...         ...        ...
2022-06-29 20:40:00   0.959328    0.958303   0.957278
2022-06-29 20:45:00   0.959320    0.958304   0.957288
2022-06-29 20:50:00   0.959359    0.958277   0.957195
2022-06-29 20:55:00   0.959426    0.958182   0.956939
2022-06-29 21:00:00   0.959405    0.958154   0.956902

[55016 rows x 3 columns]
19


In [8]:
print(SE30_index['Ratio'])
print(SE30_index.describe())

print((US2000 / SE).describe())

Date
2020-04-20 15:00:00    0.827857
2020-04-20 15:05:00    0.829645
2020-04-20 15:10:00    0.830980
2020-04-20 15:15:00    0.833099
2020-04-20 15:20:00    0.832954
                         ...   
2022-06-29 20:40:00    0.957681
2022-06-29 20:45:00    0.957021
2022-06-29 20:50:00    0.957384
2022-06-29 20:55:00    0.956787
2022-06-29 21:00:00    0.957790
Name: Ratio, Length: 55016, dtype: float64
               Open          High           Low         Close        Volume  \
count  55016.000000  55016.000000  55016.000000  55016.000000  55016.000000   
mean    2058.089842   2059.273889   2056.902317   2058.097259    101.252872   
std      249.274091    249.257222    249.279104    249.270111     52.879280   
min     1462.770000   1464.850000   1461.400000   1462.900000      1.000000   
25%     1842.795000   1843.757500   1841.837500   1842.707500     63.000000   
50%     2078.500000   2080.070000   2076.875000   2078.555000     89.000000   
75%     2277.692500   2278.682500   2276.620000

## IS and OOS split

In [45]:
SE30_index

,Open,High,Low,Close,Volume,Ratio
Date,,,,,,
2020-04-20 15:00:00,1551.22,1551.22,1542.25,1544.63,258,0.827857
2020-04-20 15:05:00,1544.75,1547.25,1540.12,1541.87,238,0.829645
2020-04-20 15:10:00,1541.75,1542.37,1537.00,1537.75,172,0.830980
2020-04-20 15:15:00,1537.87,1537.87,1531.32,1531.57,171,0.833099
2020-04-20 15:20:00,1531.44,1534.11,1529.79,1533.35,191,0.832954
...,...,...,...,...,...,...
2022-06-29 20:40:00,1901.98,1903.96,1901.36,1903.71,129,0.957681
2022-06-29 20:45:00,1903.46,1905.43,1902.71,1904.31,137,0.957021
2022-06-29 20:50:00,1904.31,1905.44,1903.93,1904.30,94,0.957384


In [46]:
US2000_index

,Open,High,Low,Close,Volume,Ratio
Date,,,,,,
2020-04-20 15:00:00,1221.94,1222.49,1217.14,1217.84,306,0.827857
2020-04-20 15:05:00,1217.79,1219.39,1216.84,1218.29,274,0.829645
2020-04-20 15:10:00,1218.14,1219.49,1216.04,1216.99,258,0.830980
2020-04-20 15:15:00,1217.09,1217.24,1214.89,1215.19,247,0.833099
2020-04-20 15:20:00,1215.09,1216.79,1213.94,1216.39,244,0.832954
...,...,...,...,...,...,...
2022-06-29 20:40:00,1734.63,1736.38,1734.33,1736.33,276,0.957681
2022-06-29 20:45:00,1736.33,1736.98,1735.03,1735.68,301,0.957021
2022-06-29 20:50:00,1735.73,1737.23,1735.73,1736.33,228,0.957384


In [51]:
part = len(SE30_index) * 0.7

SE30_index_is = SE30_index.iloc[:int(part)]
SE30_index_oos = SE30_index.iloc[int(part):]
SE30_test = SE30_index.loc['2022-05-01':'2022-05-31']

US2000_index_is = US2000_index.iloc[:int(part)]
US2000_index_oos = US2000_index.iloc[int(part):]
US2000_test = US2000_index.loc['2022-05-01':'2022-05-31']

# Pair Trading


In [117]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, TrailingStrategy
from numbers import Number
from typing import Sequence

class PairTrading(Strategy):
    # Flags for daily limit
    daily_limit = 2 #4
    daily_limit_counter = 0
    
    volume_size = 5 #5
    
    # point multiplier for contract value
    point_multiplier = 50
    prev_equity = 0
    
    # commission rate 
    number_of_trades = 0
    commission_fee = 5
    total_commission_paid = 0
    
     # Parameters for trading time
    start_trading_time = '1520' 
    end_trading_time = '2355' # 15:55 UTC+8
    
    # Margin call details
    InitialMargin = 101944*volume_size*daily_limit # minimum amount of money that a customer must deposit and maintain in their account as collateral
    MaintenanceMargin = 81555*volume_size*daily_limit # minimum amount of equity that a customer must maintain in their margin account
    ClearingHouseMargin = 76650*volume_size*daily_limit # amount threshold where broker will sell all the assets to cover the losses
    
    # Parameters for pair trading
    closed_trade_flag =  0
    bb_period = 24
    bb_upper = 2.6
    bb_lower = 2.6
    atr_period = 18
    sl = 15
    tp = 15
    
    
    
    def pay_commission(self,price):
        self._broker._cash -= price
        self.prev_equity -= price
        self.total_commission_paid += price
    
    def crossover(self,series1: Sequence, series2: Sequence) -> bool:
        # Copy from backtesting.py lib
        """
        Return `True` if `series1` just crossed over (above)
        `series2`.

            >>> crossover(self.data.Close, self.sma)
            True
        """
        series1 = (
            series1.values if isinstance(series1, pd.Series) else
            (series1, series1) if isinstance(series1, Number) else
            series1)
        series2 = (
            series2.values if isinstance(series2, pd.Series) else
            (series2, series2) if isinstance(series2, Number) else
            series2)
        try:
            return series1[-2] < series2[-2] and series1[-1] >= series2[-1]
        except IndexError:
            return False
    
    def adjust_trade_size(self):
        available_equity = self.equity*self._broker._leverage
        tradable_volume = self.equity*self._broker._leverage//(101944)
        
        if tradable_volume < self.daily_limit*self.volume_size:
            while tradable_volume < max((self.daily_limit+1)*self.volume_size,self.daily_limit*(self.volume_size+1)):
                if self.daily_limit > self.volume_size:
                    self.daily_limit -= 1
                else:
                    self.volume_size -= 1

        else:
            while tradable_volume >= min(self.daily_limit*(self.volume_size+1),(self.daily_limit+1)*self.volume_size):
                if self.daily_limit < self.volume_size:
                    self.daily_limit += 1
                    
                else:
                    self.volume_size += 1
                    
    def stop_trading(self,day):
        if self.closed_trades != None and len(self.closed_trades) >= 2 and len(self.closed_trades) != self.trading_flag_mark:
            if self.closed_trades[-1].pl < 0 and self.closed_trades[-2].pl < 0:
                self.trading_flag = True
                self.trading_flag_day = int(day)
                self.trading_flag_mark = len(self.closed_trades)
    
    def start_trading(self,day):
        if abs(self.trading_flag_day - int(day)) >= 3 and self.trading_flag:
            self.trading_flag = False
            self.trading_flag_day = 0
    
    def ratio(self):
        return self.data.Ratio
    
    def cal_buy_signals(self):
        
        return (self.data.Ratio - self.lowerBand) < 0
    
    def cal_sell_signals(self):
        return (self.data.Ratio - self.upperBand) > 0
    
    
    def init(self):
        # super.init()
        super().init()
        # super().set_atr_periods(self.ATR_period)
        # super().set_trailing_sl(self.ATR_sl)
        self.ratio_signal = self.I(self.ratio)
        self.upperBand, self.middleBand, self.lowerBand = self.I(talib.BBANDS, self.data.Ratio , timeperiod=self.bb_period, nbdevup=self.bb_upper, nbdevdn=self.bb_lower, matype=0,plot=False)
        self.buy_signals = self.I(self.cal_buy_signals,plot=False)
        # print((self.buy_signals == True).sum())
        # print((self.buy_signals == False).sum())
        self.sell_signals = self.I(self.cal_sell_signals,plot=False)
        self.SMA = self.I(talib.SMA, self.data.Close, timeperiod=self.atr_period)
        
        self.prev_equity = self.equity
        self.lastday = self.data.index.values[-1]

    def next(self):
        super().next()
        # Code to get date and time
        date_str = np.datetime_as_string(self.data.index.values[-1], unit='D')
        date_obj = pd.to_datetime(date_str)
        day = date_obj.strftime('%Y-%m-%d')[-2:]
        
        ts = pd.to_datetime(str(self.data.index.values[-1].copy())) 
        d = ts.strftime('%Y%m%d%H%M%S')[-6:-2]
        
        if self.equity*self._broker._leverage < self.InitialMargin:
            print("You do not have the sufficient funds to trade")
            # self.adjust_trade_size()
        
        
        else:
        
            if (day in ['29','30','31']) and self.expiration_flag == False:
                self.expiration_flag = True
            
                if self.position:
                    self.position.close()
                    
                # we would like to not trade on the last few days of the months as the market would be volatile and alot of individuals would start to sell their futures
            elif (self.equity*self._broker._leverage < self.ClearingHouseMargin) and (self.equity*self._broker._leverage > self.MaintenanceMargin):
                print("You are in the maintenance margin zone, you should top up your account")
                # can consider reducing the volume size to a lower volume to reduce the risk of margin call
            
            else:
                
                self.expiration_flag = False

                # Start trading based on trading time
                if (d >= self.start_trading_time) and (d <= self.end_trading_time) and (self.equity*self._broker._leverage > self.MaintenanceMargin) :
                    
                    # Trading logic
                    # Since ratio is calculated by the ratio of the two futures, we would like to buy the first future and sell the second future if ratio is low
                    if self.buy_signals[-1] == True and self.daily_limit>self.daily_limit_counter:
                        self.sell(size=self.volume_size,sl=self.data.Close[-1]+self.sl, tp= self.data.Close[-1]-self.tp)#(self.data.Close[-1]*self.data.Ratio[-1] - (self.data.Close[-1]*self.data.Ratio[-1]))/2 )
                        self.number_of_trades += 1
                        self.daily_limit_counter += 1
                        self.pay_commission(self.commission_fee)
                        self.closed_trade_flag = self.middleBand[-1]
                        # print("Buy signal")
                        
                        
                    if self.sell_signals[-1] == True and self.daily_limit>self.daily_limit_counter:
                        self.buy(size=self.volume_size, sl = self.data.Close[-1]-self.sl, tp = self.data.Close[-1]+self.tp)
                        self.number_of_trades += 1
                        self.daily_limit_counter += 1
                        self.pay_commission(self.commission_fee)
                        self.closed_trade_flag = self.middleBand[-1]
                        # print("Sell signal")
                    
                    
                    if abs(self.ratio_signal[-1] - self.closed_trade_flag) == 0:
                            if self.position:
                                self.position.close()
                    
                    # Stop loss can be done by checking if the spread deviative more form the mean or if the cointegration value is broken and the pair is not cointegrated anymore
                    
                    # Take profit can be done by checking if the spread is close to the mean
                        
        
        # Need to sell all position at the end of the day
        if d > '2300':
            self.daily_limit_counter = 0
            if self.position:
                self.position.close()
        
        
        #Check everytime whether there is a change in equity
        if self.prev_equity != self.equity:
            self._broker._cash += (self.equity - self.prev_equity)*(self.point_multiplier-1)
            self.prev_equity = self.equity
            
            while (len(self.closed_trades) - self.number_of_trades) > 0:
                self.pay_commission(self.commission_fee)
                self.number_of_trades += 1
            
        # self.adjust_trade_size()
        # self.stop_trading(day)
        # self.start_trading(day)
        
        if self.data.index[-1] == self.lastday:
            print(f"this is the volume: {self.volume_size} and this is the daily limit: {self.daily_limit}")
            print(d)
        #     print((self.buy_signals == True).sum())
        #     print((self.buy_signals == False).sum())
            
            # print("# of closed trade: ",self.count_closed_trades,"Total number of sl and tp hit: ", self.count_sl_tp, "# of sl: ", self.count_sl, "# of tp: ", self.count_tp)

# Back Test Result

In [118]:
from backtesting.lib import compute_stats

bt = Backtest(SE30_index_is, PairTrading, cash=1250000, margin=1,exclusive_orders=True)
stats = bt.run()
stats

this is the volume: 5 and this is the daily limit: 2
1805


Start                     2020-04-20 15:00:00
End                       2021-10-27 18:05:00
Duration                    555 days 03:05:00
Exposure Time [%]                   34.966633
Equity Final [$]                    1303125.0
Equity Peak [$]                    1303759.85
Return [%]                               4.25
Buy & Hold Return [%]               50.999268
Return (Ann.) [%]                    2.520119
Volatility (Ann.) [%]                2.490509
Sharpe Ratio                         1.011889
Sortino Ratio                        1.590523
Calmar Ratio                         0.865182
Max. Drawdown [%]                   -2.912819
Avg. Drawdown [%]                   -0.124118
Max. Drawdown Duration      251 days 19:40:00
Avg. Drawdown Duration        5 days 12:16:00
# Trades                                  665
Win Rate [%]                         50.37594
Best Trade [%]                       0.990008
Worst Trade [%]                     -1.641505
Avg. Trade [%]                    

In [119]:
bt_oos = Backtest(SE30_index_oos, PairTrading, cash=1250000, margin=1, exclusive_orders=True)
stats_oos = bt_oos.run()
stats_oos

this is the volume: 5 and this is the daily limit: 2
2100


Start                     2021-10-27 18:10:00
End                       2022-06-29 21:00:00
Duration                    245 days 02:50:00
Exposure Time [%]                   30.342321
Equity Final [$]                    1308315.0
Equity Peak [$]                    1313176.95
Return [%]                             4.6652
Buy & Hold Return [%]              -18.342731
Return (Ann.) [%]                    7.019634
Volatility (Ann.) [%]                3.671949
Sharpe Ratio                         1.911691
Sortino Ratio                         3.07225
Calmar Ratio                          3.89988
Max. Drawdown [%]                   -1.799961
Avg. Drawdown [%]                   -0.167026
Max. Drawdown Duration       47 days 21:10:00
Avg. Drawdown Duration        2 days 08:20:00
# Trades                                  287
Win Rate [%]                        52.961672
Best Trade [%]                       0.800034
Worst Trade [%]                     -1.313909
Avg. Trade [%]                    

# Plotting results

In [120]:
bt.plot(plot_equity=True,plot_drawdown=True)

/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p30829', ...)

In [ ]:
bt_oos.plot(plot_equity=True,plot_drawdown=True)

## Analyse on 1month 5min candle

In [112]:
bt = Backtest(SE30_test, PairTrading, cash=1250000, margin=1,exclusive_orders=True )
stats = bt.run()
stats
bt.plot(plot_equity=True,plot_drawdown=True)

this is the volume: 5 and this is the daily limit: 2
2315


/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p26985', ...)

# Optimization

In [116]:
from bayes_opt import BayesianOptimization

def optimize(bb_period,bb, atr_period, sl, volume_size, daily_limit,tp):
    
    PT = PairTrading
    
    PT.bb_period = int(bb_period)
    PT.bb_lower = int(bb)
    PT.bb_upper = int(bb)
    # DT_MT5.volume_size = int(volume_size)
    # DT_MT5.daily_limit = int(daily_limit)
    
    PT.atr_period = int(atr_period)
    PT.sl = int(sl)
    PT.volume_size = int(volume_size)
    PT.daily_limit = int(daily_limit)
    PT.tp = int(tp)
    
    bt = Backtest(SE30_index_is, PT, cash=1250000, margin=1 , exclusive_orders=True)
    stats = bt.run()
    return stats['Equity Final [$]'] 

PT_bo = BayesianOptimization(
    optimize,
    {
        # 'k1': (1, 125),
        # "k2": (1, 125),
        "volume_size": (3, 5),
        # "N": (3, 10),
        "daily_limit": (1, 2),
        "atr_period": (2, 20),
        # "atr_tp": (2, 12),
        # "atr_sl": (1, 5),
        "sl": (1, 20),
        
        'bb_period': (2, 30),
        'bb': (1, 5),
        'tp': (1, 30),
    }
)

PT_bo.maximize(
    init_points=10,
    n_iter=200
    
    )

print(PT_bo.max)

|   iter    |  target   | atr_pe... |    bb     | bb_period | daily_... |    sl     |    tp     | volume... |
-------------------------------------------------------------------------------------------------------------
this is the volume: 3 and this is the daily limit: 1
1805
| 1         | 1.219e+06 | 10.55     | 1.088     | 2.398     | 1.126     | 12.63     | 3.669     | 3.533     |
this is the volume: 3 and this is the daily limit: 1
1805
| 2         | 1.235e+06 | 10.04     | 2.435     | 5.0       | 1.58      | 11.09     | 14.41     | 3.077     |
this is the volume: 4 and this is the daily limit: 1
1805
| 3         | 1.262e+06 | 9.846     | 2.444     | 8.159     | 1.854     | 15.42     | 25.89     | 4.272     |
this is the volume: 3 and this is the daily limit: 1
1805
| 4         | 1.285e+06 | 11.02     | 2.166     | 22.1      | 1.079     | 1.138     | 9.159     | 3.485     |
this is the volume: 4 and this is the daily limit: 1
1805
| 5         | 1.255e+06 | 18.26     | 3.968     | 